In [1]:
import sys
sys.path.append("/home/alok/Programming/py-manifolds")

In [2]:
import math

import jax
import numpy as np

from manifolds.manifold import Tensor, Tangent, Cotangent
from manifolds.sphere import Sphere, StereographicChart, SpherePoint

In [3]:
sphere = Sphere(5)

In [4]:
northpole = SpherePoint(np.array([0., 0., 5.]))
southpole = SpherePoint(np.array([0., 0., -5.]))

In [5]:
south_projection = sphere.preferred_chart(northpole)
north_projection = sphere.preferred_chart(southpole)

In [6]:
northern_hemi_point = SpherePoint(np.array([0., 3., 4.]))
print(south_projection.point_to_coords(northern_hemi_point))
print(north_projection.point_to_coords(northern_hemi_point))
print(
    south_projection.coords_to_point(
        south_projection.point_to_coords(northern_hemi_point)
    ).coords
)

[0.         1.66666667]
[ 0. 15.]
[0. 3. 4.]


In [7]:
def south_to_north(c):
    return north_projection.point_to_coords(south_projection.coords_to_point(c))

In [8]:
jacobian = jax.jacfwd(south_to_north)

In [9]:
jacobian(south_projection.point_to_coords(northern_hemi_point)) @ np.array([1, 1])

DeviceArray([ 8.999998, -8.999992], dtype=float32)

In [10]:
15 / 1.6667

8.999820003599927

In [11]:
pt, tangent = jax.jvp(south_to_north, [south_projection.point_to_coords(northern_hemi_point)], [np.array([1., 1.])])

In [12]:
tangent

DeviceArray([ 8.999998, -8.999992], dtype=float32)

In [13]:
t = Tangent(south_projection, south_projection.point_to_coords(northern_hemi_point), np.array([1., 1.]))
print(t.to_chart(north_projection))

Tangent(chart=<manifolds.sphere.StereographicChart object at 0x7fa875e1bb20>, p_coords=DeviceArray([ 0.      , 14.999995], dtype=float32), v_coords=DeviceArray([ 8.999998, -8.999992], dtype=float32))


In [14]:
ct = Cotangent(south_projection, south_projection.point_to_coords(northern_hemi_point), np.array([1., 1.]))
print(ct.to_chart(north_projection))

Cotangent(chart=<manifolds.sphere.StereographicChart object at 0x7fa875e1bb20>, p_coords=DeviceArray([ 0.      , 15.000001], dtype=float32), v_coords=(DeviceArray([ 0.11111111, -0.1111111 ], dtype=float32),))


In [15]:
t_tensor = Tensor(south_projection, south_projection.point_to_coords(northern_hemi_point), np.array([1., 2.]), n_contra=0)
print(t_tensor.to_chart(north_projection))

Tensor(chart=<manifolds.sphere.StereographicChart object at 0x7fa875e1bb20>, p_coords=DeviceArray([ 0.      , 15.000001], dtype=float32), t_coords=DeviceArray([  8.999998, -17.999985], dtype=float32), n_contra=0)


In [16]:
ct_tensor = Tensor(south_projection, south_projection.point_to_coords(northern_hemi_point), np.array([3., 1.]), n_contra=1)
print(ct_tensor.to_chart(north_projection))

Tensor(chart=<manifolds.sphere.StereographicChart object at 0x7fa875e1bb20>, p_coords=DeviceArray([ 0.      , 15.000001], dtype=float32), t_coords=DeviceArray([ 0.3333333 , -0.11111104], dtype=float32), n_contra=1)


In [17]:
t_tensor.tensorprod(ct_tensor).to_chart(north_projection)

Tensor(chart=<manifolds.sphere.StereographicChart object at 0x7fa875e1bb20>, p_coords=DeviceArray([ 0.      , 15.000001], dtype=float32), t_coords=DeviceArray([[ 2.9999993, -5.9999948],
             [-0.9999991,  1.999997 ]], dtype=float32), n_contra=1)

In [18]:
t_tensor.to_chart(north_projection).tensorprod(ct_tensor.to_chart(north_projection))

Tensor(chart=<manifolds.sphere.StereographicChart object at 0x7fa875e1bb20>, p_coords=DeviceArray([ 0.      , 15.000001], dtype=float32), t_coords=DeviceArray([[ 2.9999993, -5.9999948],
             [-0.9999991,  1.999997 ]], dtype=float32), n_contra=1)